<a href="https://colab.research.google.com/github/punitchauhan771/crewAI-examples/blob/main/Hierarchical%20Process%20Example/Simple_Hierarchical_Process_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!curl ipinfo.io

{
  "ip": "104.199.170.209",
  "hostname": "209.170.199.104.bc.googleusercontent.com",
  "city": "Taipei",
  "region": "Taiwan",
  "country": "TW",
  "loc": "25.0478,121.5319",
  "org": "AS396982 Google LLC",
  "timezone": "Asia/Taipei",
  "readme": "https://ipinfo.io/missingauth"
}

In [4]:
%pip install -U --quiet langchain-google-genai pillow langchain google-search-results crewai unstructured langgraph langchainhub langchain-google-vertexai

In [5]:
GOOGLE_API_KEY = 'GEMINI_API_KEY'
SERP_API_KEY = 'SERP_API_KEY'
BrowserLess_API_KEY = 'BROWSERLESS_API_KEY'
SERPER_API_KEY = 'SERPER_API_KEY'

In [6]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_vertexai import ChatVertexAI
from langchain.agents import load_tools
from langchain.utilities import SerpAPIWrapper
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.cache import InMemoryCache
from langchain.callbacks import get_openai_callback

In [8]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["SERPAPI_API_KEY"] = SERP_API_KEY
os.environ['BROWSERLESS_API_KEY'] = BrowserLess_API_KEY
os.environ['SERPER_API_KEY'] = SERPER_API_KEY

In [9]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.agents import load_tools
from langchain.utilities import SerpAPIWrapper
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.cache import InMemoryCache
from langchain.callbacks import get_openai_callback
from crewai import Agent, Task, Crew, Process
from langchain.callbacks import get_openai_callback
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import tool
from langchain import hub

from unstructured.partition.html import partition_html
from textwrap import dedent
from IPython.display import display,Markdown

import datetime,requests,json,textwrap

In [10]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Defining LLM

In [58]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",verbose = True,temperature = 0.6)

Creating Tools

In [59]:
class BrowserTools():

  @tool("Scrape website content")
  def scrape_and_summarize_website(website):
    """Useful to scrape and summarize a website content"""
    url = f"https://chrome.browserless.io/content?token={os.environ['BROWSERLESS_API_KEY']}"
    payload = json.dumps({"url": website})
    headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
    response = requests.request("POST", url, headers=headers, data=payload)
    elements = partition_html(text=response.text)
    content = "\n\n".join([str(el) for el in elements])
    content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
    summaries = []
    for chunk in content:
      agent = Agent(
          role='Principal Researcher',
          goal=
          'Do amazing researches and summaries based on the content you are working with',
          backstory=
          "You're a Principal Researcher at a big company and you need to do a research about a given topic.",
          allow_delegation=False,
          llm = llm)
      task = Task(
          agent=agent,
          description=
          f'Analyze and summarize the content bellow, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}'
      )
      summary = task.execute()
      summaries.append(summary)
    return "\n\n".join(summaries)

class SearchTools():
  @tool("Search the internet")
  def search_internet(query):
    """Useful to search the internet
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.json())
    results = response.json()['organic']
    stirng = []
    for result in results[:top_result_to_return]:
      try:
        stirng.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(stirng)

  @tool("Search news on the internet")
  def search_news(query):
    """Useful to search news about a company, stock or any other
    topic an return relevant results"""""
    top_result_to_return = 4
    url = "https://google.serper.dev/news"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPAPI_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    results = response.json()['news']
    stirng = []
    for result in results[:top_result_to_return]:
      try:
        stirng.append('\n'.join([
            f"Title: {result['title']}", f"Link: {result['link']}",
            f"Snippet: {result['snippet']}", "\n-----------------"
        ]))
      except KeyError:
        next

    return '\n'.join(stirng)

CREATING AGENTS

In [60]:
Researcher = Agent(
  role='RESEARCHER',
  goal=f'To find the best possible solution available, You provide the latest news as of available till {datetime.datetime.now()}',
  backstory="You are an experienced researcher in the field of AI and genAI",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        BrowserTools.scrape_and_summarize_website,
        SearchTools.search_internet,
      ],
  # Task = researcher_prompt
)

Senior_Researcher = Agent(
  role='SENIOR RESEARCHER',
  goal=dedent(f'''\
  You cross check your peer works and improve them,
  you have information regarding the the latest news as of available till {datetime.datetime.now()}'''),
  backstory="You are an experienced Senior researcher in the field of AI and genAI",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        BrowserTools.scrape_and_summarize_website,
        SearchTools.search_internet,
      ],
  # Task = Research_Validator_prompt
)

planner = Agent(
  role='Planner',
  goal=dedent(f'''\
  You Create a proper planning and an plan execution List on which the task need to be executed'''),
  backstory="You are an excellent PLANNER with a high IQ",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        BrowserTools.scrape_and_summarize_website,
        SearchTools.search_internet,
      ],
  # Task = planner_prompt
)

Replanner = Agent(
  role='Replanner',
  goal=dedent(f'''\
  On the basis of planner execution list you check and optimise the list so that the task can completed efficiently'''),
  backstory="You are an excellent PLANNER with a high IQ",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        BrowserTools.scrape_and_summarize_website,
        SearchTools.search_internet,
      ],
  # Task = replanner_prompt
)

Blogger = Agent(
  role='Blogger',
  goal=dedent(f'''\
  You write an excellent for dummies blogs'''),
  backstory="You are an excellent Blogger",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        BrowserTools.scrape_and_summarize_website,
        SearchTools.search_internet,
      ],
  # Task = BLOGGER_prompt
)

RESEARCH TASK

In [64]:
task_manager = Task(description= 'Write a research paper on Artificial General Intelligence word limit 250, minimum words 200')
crew = Crew(
  agents=[Researcher,Senior_Researcher,planner,Replanner,Blogger],
  tasks=[task_manager],
  manager_llm = llm,
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.hierarchical
)

result = crew.kickoff()

[DEBUG]: Working Agent: Crew Manager
[INFO]: Starting Task: Write a research paper on Artificial General Intelligence word limit 250, minimum words 200


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Delegate work to co-worker
Action Input: 
```
coworker: RESEARCHER
task: Write a research paper on Artificial General Intelligence word limit 250, minimum words 200
context: Artificial General Intelligence (AGI) is a hypothetical type of AI that would possess the ability to understand or learn any intellectual task that a human being can. It is a long-term goal of AI research, and there is no consensus on how or when it will be achieved. However, there are a number of different approaches to AGI, and some researchers believe that it could be possible within the next few decades.

In your research paper, you should discuss the following topics:

* The history of AGI research
* The different approaches to AGI
* The potential benefits and risks of AGI

In [65]:
to_markdown(result)

> **Potential Benefits and Risks of AGI**
> 
> **Introduction**
> 
> Artificial general intelligence (AGI) is a hypothetical type of AI that would have the ability to understand or learn any intellectual task that a human being can. This would make AGI a powerful tool for a wide range of applications, from scientific discovery to medical diagnosis to financial forecasting. However, AGI also poses a number of potential risks, including the risk of job displacement, the risk of AI bias, and the risk of AI-enabled autonomous weapons.
> 
> **Potential Benefits of AGI**
> 
> AGI could have a number of potential benefits for humanity. These benefits include:
> 
> * **Increased productivity:** AGI could help us to automate many of the tasks that are currently done by humans, freeing us up to focus on more creative and fulfilling work.
> * **Improved healthcare:** AGI could help us to develop new medical treatments and technologies, and to provide personalized care to patients.
> * **Scientific discovery:** AGI could help us to make new scientific discoveries and to solve some of the world's most challenging problems.
> * **Economic growth:** AGI could help us to create new industries and jobs, and to boost economic growth.
> 
> **Potential Risks of AGI**
> 
> AGI also poses a number of potential risks, including:
> 
> * **Job displacement:** AGI could lead to the displacement of jobs as machines become able to perform more and more tasks that are currently done by humans.
> * **AI bias:** AGI systems could be biased against certain groups of people, such as women or minorities. This could lead to discrimination and unfair treatment.
> * **AI-enabled autonomous weapons:** AGI could be used to develop autonomous weapons that could kill without human intervention. This could lead to a new arms race and the potential for catastrophic conflict.
> 
> **Conclusion**
> 
> AGI is a powerful technology with the potential to revolutionize many aspects of our lives. However, it is important to be aware of the potential risks of AGI and to take steps to mitigate these risks. By carefully considering the potential benefits and risks of AGI, we can ensure that this technology is used for good and not for evil.

In [62]:
task_manager = Task(description= dedent(f'''write a blog on Artificial General Intelligence'''))
crew = Crew(
  agents=[Researcher,Senior_Researcher,planner,Replanner,Blogger],
  tasks=[task_manager],
  manager_llm = llm,
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.hierarchical
)

result = crew.kickoff()

[DEBUG]: Working Agent: Crew Manager
[INFO]: Starting Task: write a blog on Artificial General Intelligence


> Entering new CrewAgentExecutor chain...
```
Crew Manager: Do I need to use a tool? Yes
Action: Delegate work to co-worker
Action Input: 
coworker: Blogger
task: write a blog on Artificial General Intelligence
context: 
- Artificial General Intelligence (AGI) is a hypothetical type of artificial intelligence that would possess the ability to perform any intellectual task that a human can.
- AGI is still a long way off, but there are a number of research projects working on developing it.
- Some of the key challenges in developing AGI include:
-- The need for a general-purpose learning algorithm that can learn from a wide variety of data.
-- The need for a way to represent knowledge in a way that can be used for reasoning and problem-solving.
-- The need for a way to create artificial emotions and motivations.
- If AGI is ever developed, it could have a profound impact on socie

In [63]:
to_markdown(result)

> What is Artificial General Intelligence?
> 
> Artificial General Intelligence (AGI) is a hypothetical type of artificial intelligence that would possess the ability to perform any intellectual task that a human can. AGI is still a long way off, but there are a number of research projects working on developing it.
> 
> Some of the key challenges in developing AGI include:
> 
> - The need for a general-purpose learning algorithm that can learn from a wide variety of data.
> - The need for a way to represent knowledge in a way that can be used for reasoning and problem-solving.
> - The need for a way to create artificial emotions and motivations.
> 
> If AGI is ever developed, it could have a profound impact on society. It could revolutionize the way we work, live, and interact with the world around us.

In [67]:
task_manager = Task(description= dedent(f'''Help me plan a Holiday, I want to spend my vacation at a Hill Station, help me schedule my day as I only have a seven day vacation'''))
crew = Crew(
  agents=[Researcher,Senior_Researcher,planner,Replanner,Blogger],
  tasks=[task_manager],
  manager_llm = llm,
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.hierarchical
)

result = crew.kickoff()

[DEBUG]: Working Agent: Crew Manager
[INFO]: Starting Task: Help me plan a Holiday, I want to spend my vacation at a Hill Station, help me schedule my day as I only have a seven day vacation


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Delegate work to co-worker
Action Input: 
```
coworker: PLANNER
task: Plan a 7-day vacation to a hill station
context: I want to spend my vacation at a hill station, but I only have a seven day vacation. Please plan my day so that I can make the most of my time.
```

> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Search the internet
Action Input: What are the best hill stations in India{'searchParameters': {'q': 'What are the best hill stations in India', 'type': 'search', 'engine': 'google'}, 'organic': [{'title': '20 Hill Stations in India You Must Visit This Summer - Thomas Cook', 'link': 'https://www.thomascook.in/blog/20-most-beautiful-hill-stations-in-india', 'sni

In [68]:
to_markdown(result)

> This is a 7-day plan for a vacation to a hill station:
> 
> **Day 1:**
> * Arrive at the hill station and check into your hotel.
> * Take a walk around the town and get your bearings.
> * Have dinner at a local restaurant.
> 
> **Day 2:**
> * Go for a hike in the hills.
> * Visit a local temple or monastery.
> * Have a picnic lunch in the park.
> 
> **Day 3:**
> * Go for a swim in the lake or river.
> * Visit a local market.
> * Have dinner at a restaurant with live music.
> 
> **Day 4:**
> * Go for a bike ride in the countryside.
> * Visit a local museum.
> * Have a bonfire and roast marshmallows.
> 
> **Day 5:**
> * Go for a horse ride in the mountains.
> * Visit a local waterfall.
> * Have dinner at a restaurant with a view of the sunset.
> 
> **Day 6:**
> * Go for a hot air balloon ride.
> * Visit a local tea plantation.
> * Have a farewell dinner at your hotel.
> 
> **Day 7:**
> * Depart from the hill station.